In [1]:
import cv2
import pyautogui
import mediapipe as mp
import socket
import time

# Host and port to listen on
host = '0.0.0.0'  # Listen on all available interfaces
port = 12345      # Choose a port number above 1024

# Create a TCP/IP socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
# Bind the socket to the address and port
server_socket.bind((host, port))
# Listen for incoming connections
server_socket.listen(1)
print("Server is listening on", host, "port", port)

# Accept incoming connection
client_socket, client_address = server_socket.accept()

print("Connection established with", client_address)


cam = cv2.VideoCapture(0)
screen_w, screen_h = pyautogui.size()
face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)

while True:
    _, frame = cam.read()
    frame_h, frame_w, _ = frame.shape
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    output = face_mesh.process(rgb_frame)
    landmark_points = output.multi_face_landmarks
    if landmark_points:
        landmarks = landmark_points[0].landmark
        left = [landmarks[145], landmarks[159]]
        for id, landmark in enumerate(landmarks[474:478]):
            x = int(landmark.x * frame_w)
            y = int(landmark.y * frame_h)
            cv2.circle(frame, (x, y), 3, (0, 255, 0))
            if id == 1:
                screen_x = screen_w * landmark.x
                screen_y = screen_h * landmark.y
                processed_data = str(screen_x)+" "+str(screen_y)+" "+str(left[0].y)+" "+str(left[1].y)
                print(processed_data)
                # Send data to client
                client_socket.sendall(processed_data.encode())
                time.sleep(0.1)
    cv2.imshow('Eye Controlled Mouse', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all OpenCV windows
cam.release()
cv2.destroyAllWindows()

# Close the connection
client_socket.close()
server_socket.close()

Server is listening on 0.0.0.0 port 12345
Connection established with ('192.168.1.40', 62077)
1026.4770126342773 250.63260555267334 0.2571873664855957 0.23566842079162598
1025.9932708740234 248.30870747566223 0.2555907368659973 0.23315533995628357
1024.115982055664 249.0831470489502 0.2560209035873413 0.23351317644119263
1027.0154571533203 249.81374859809875 0.2557409703731537 0.23390820622444153
1025.3045654296875 249.26625609397888 0.25656408071517944 0.23230955004692078
1023.6979293823242 250.30208230018616 0.25659406185150146 0.23438343405723572
991.5718460083008 243.47019016742706 0.259740948677063 0.2383030354976654
935.9731292724609 238.8198834657669 0.26148512959480286 0.24024809896945953
932.7107620239258 240.21193385124207 0.26431742310523987 0.24109306931495667
947.8912353515625 241.21068120002747 0.262748122215271 0.2398335039615631
977.2890472412109 243.90078127384186 0.25731849670410156 0.23531073331832886
1007.3748779296875 248.79713773727417 0.25633326172828674 0.232246